In [1]:
import requests
import os

# Define a function to search for photos on Flickr
def search_flickr(query, api_key, num_images=10):
    #Base URL for the Flickr API
    url = "https://api.flickr.com/services/rest/"

    #Parameters for the API request
    params = {
        "method": "flickr.photos.search",
        "api_key": api_key,
        "text": query,
        "tags": "computer setup",
        "sort": "relevance",
        "per_page": num_images,
        "format": "json",
        "nojsoncallback": 1
    }

    # Send a GET request to the Flickr API
    response = requests.get(url, params=params)

    #Check if the request was successful
    if response.status_code == 200:
        #Parse JSON response and extract the list of photos
        return response.json()['photos']['photo']
    else:
        #Print error message if request failed
        print("Failed to fetch data from Flicker API.")
        return []

#Define a function to construct the URL of a photo
def get_photo_url(photo, size='large'):
    #Base URL for the photo
    url = f"https://live.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}"

    #Append size suffix to the URL based on the specified size

    if size == 'large':
        url +='_b.jpg'
    elif size =="medium":
        url+="_z.jpg"
    else:
        url += '.jpg'

    return url
    
def download_images(photo_list, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

    for i, photo in enumerate(photo_list):
        img_url = get_photo_url(photo)

        img_path = os.path.join(directory, f"image_{i+1}.jpg")

        with open(img_path, 'wb') as img_file:
            img_file.write(requests.get(img_url).content)

#Define the main function
def main():
    query= "computer"
    api_key= "5946546e4038b015cb4f45a9f30ef0d9"
    num_images= 100
    photo_list = search_flickr(query, api_key, num_images)
    download_images(photo_list, "computer_images")


if __name__ == "__main__":
    main()



In [7]:
import os
from glob import glob
import pandas as pd
from xml.etree import ElementTree as et
from functools import reduce

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
xmlfiles = glob('yolo/data_images/*.xml')

In [6]:
xmlfiles

['yolo/data_images\\balloon_image_1.xml',
 'yolo/data_images\\balloon_image_10.xml',
 'yolo/data_images\\balloon_image_100.xml',
 'yolo/data_images\\balloon_image_11.xml',
 'yolo/data_images\\balloon_image_12.xml',
 'yolo/data_images\\balloon_image_13.xml',
 'yolo/data_images\\balloon_image_14.xml',
 'yolo/data_images\\balloon_image_15.xml',
 'yolo/data_images\\balloon_image_16.xml',
 'yolo/data_images\\balloon_image_17.xml',
 'yolo/data_images\\balloon_image_18.xml',
 'yolo/data_images\\balloon_image_19.xml',
 'yolo/data_images\\balloon_image_2.xml',
 'yolo/data_images\\balloon_image_20.xml',
 'yolo/data_images\\balloon_image_21.xml',
 'yolo/data_images\\balloon_image_22.xml',
 'yolo/data_images\\balloon_image_23.xml',
 'yolo/data_images\\balloon_image_24.xml',
 'yolo/data_images\\balloon_image_25.xml',
 'yolo/data_images\\balloon_image_26.xml',
 'yolo/data_images\\balloon_image_27.xml',
 'yolo/data_images\\balloon_image_28.xml',
 'yolo/data_images\\balloon_image_29.xml',
 'yolo/data_

In [ ]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    #extract filename
    image_name = root.find('filename'.text)
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []

    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])

    return parser